<a href="https://colab.research.google.com/github/ElnathanTiokou/Adversarial_Robustness_of_FL_systems/blob/main/Tensorflow_federated_Robustness/dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from typing import Callable, Tuple

import os
import tensorflow as tf
import numpy as np
from attacks.data_attacker import AbstractDataAttacker
from attacks.model_attacker import BackdoorAttack
from constants import *



emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()

class Dataset():

    def __init__(self):
        self.partitioned_data = None
        self.data_preprocessor = None

    def preprocess(self, dataset):
      def batch_format_fn(element):
        """Flatten a batch of EMNIST data and return a (features, label) tuple."""
        return (tf.reshape(element['pixels'], [-1, 784]), 
                tf.reshape(element['label'], [-1, 1]))
      self.data_preprocessor = dataset.batch(BATCH_SIZE).map(batch_format_fn)
      return self.data_preprocessor

    # We now select a small number of clients, and apply the preprocessing above to their datasets.
    def partition_dataset(self, non_iid_deg: float):
        if self.x_train is None:
            return
        client_ids = sorted(emnist_train.client_ids)[:NUM_CLIENTS]
        self.partitioned_data = [self.preprocess(emnist_train.create_tf_dataset_for_client(x))
      for x in client_ids
    ]

    def attack_data(self, data_attacker: AbstractDataAttacker):
        if data_attacker is not None:
            self.partitioned_data = data_attacker.attack(self.partitioned_data)

    def create_test_data(self):
        
        return emnist_test

    def backdoor(self, model_attacker: BackdoorAttack):
        self.partitioned_data = model_attacker.attack_train(self.partitioned_data)
        self.x_test, self.y_test = model_attacker.attack_test(self.x_test, self.y_test)

